## C++ Posterior function performance & correctness evaluation

***

In [5]:
import unittest
import numpy as np
import pandas as pd
import ctypes
from customProphet6 import *

class TestLogPosterior(unittest.TestCase):
    def setUp(self):
        # Load the shared library
        self.lib = ctypes.CDLL('./libminus_log_posterior_5.so')
        self.lib.minus_log_posterior.argtypes = [
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            ctypes.c_size_t,
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            ctypes.c_size_t,
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            ctypes.c_size_t,
            ctypes.c_double,
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            ctypes.c_size_t,
            ctypes.c_double,
            ctypes.c_double,
            ctypes.c_double,
            ctypes.c_double,
            ctypes.c_double
        ]
        self.lib.minus_log_posterior.restype = ctypes.c_double

        # Load data
        df = pd.read_csv('peyton_manning.csv')

        # Instantiate & initialize a model
        self.model = CustomProphet()
        self.model.y = df['y'].values
        if df['ds'].dtype != 'datetime64[ns]':
            self.model.ds = pd.to_datetime(df['ds'])
        else:
            self.model.ds = df['ds']

        self.model.t_scaled = np.array((self.model.ds - self.model.ds.min()) / (self.model.ds.max() - self.model.ds.min()))
        self.model.T = df.shape[0]

        self.model.scale_period = (self.model.ds.max() - self.model.ds.min()).days
        self.model._normalize_y()
        self.model._generate_change_points()

        self.params = np.ones((47,))
    
    def tearDown(self):
        # This method is called after each test
        print('Test passed: posterior function correctly implemented')

    def test_minus_log_posterior(self):
        # Convert data to ctypes
        params_ctypes = np.ascontiguousarray(self.params, dtype=np.float64)
        t_scaled_ctypes = np.ascontiguousarray(self.model.t_scaled, dtype=np.float64)
        change_points_ctypes = np.ascontiguousarray(self.model.change_points, dtype=np.float64)
        normalized_y_ctypes = np.ascontiguousarray(self.model.normalized_y, dtype=np.float64)

        # Call the Python method
        mpl_python = self.model._minus_log_posterior(self.params)

        # Call the C++ function
        mlp_cpp = self.lib.minus_log_posterior(
            params_ctypes, len(params_ctypes),
            t_scaled_ctypes, len(t_scaled_ctypes),
            change_points_ctypes, len(change_points_ctypes),
            self.model.scale_period,
            normalized_y_ctypes, len(normalized_y_ctypes),
            self.model.sigma_obs,
            self.model.sigma_k,
            self.model.sigma_m,
            self.model.sigma,
            self.model.tau
        )

        # Assert that the values are close
        self.assertAlmostEqual(mpl_python, mlp_cpp, places=5, msg="The C++ and Python minus log posterior values do not match.")

# Run in a jupyter notebook
suite = unittest.TestLoader().loadTestsFromTestCase(TestLogPosterior)

# Run the test suite
unittest.TextTestRunner(verbosity=2).run(suite)

# Run in a script
#if __name__ == '__main__':
 #   unittest.main()

test_minus_log_posterior (__main__.TestLogPosterior.test_minus_log_posterior) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.034s

OK


Test passed: posterior function correctly implemented


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [1]:
import numpy as np
import pandas as pd
import ctypes
from customProphet6 import *

# Load the shared library
lib = ctypes.CDLL('./libminus_log_posterior_5.so')

# Define argument and return types
lib.minus_log_posterior.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double
]
lib.minus_log_posterior.restype = ctypes.c_double

# Load data
df = pd.read_csv('peyton_manning.csv')

# Instantiate & initialize a model
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

params = np.ones((47,))

# Convert data to ctypes
params_ctypes = np.ascontiguousarray(params, dtype=np.float64)
t_scaled_ctypes = np.ascontiguousarray(model.t_scaled, dtype=np.float64)
change_points_ctypes = np.ascontiguousarray(model.change_points, dtype=np.float64)
normalized_y_ctypes = np.ascontiguousarray(model.normalized_y, dtype=np.float64)

mpl = model._minus_log_posterior(params)

# Call the C++ function
mlp_cpp = lib.minus_log_posterior(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau
)

print("Minus log posterior (C++):", mlp_cpp)
print("Minus log posterior (Python):", mpl)

Minus log posterior (C++): 320689450.8123412
Minus log posterior (Python): 320689450.8123411


In [2]:
%%timeit -r 50 -n 50

lib.minus_log_posterior(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau
)

307 µs ± 41.4 µs per loop (mean ± std. dev. of 50 runs, 50 loops each)


In [3]:
%%timeit -r 50 -n 50

model._minus_log_posterior(params)

1.08 ms ± 363 µs per loop (mean ± std. dev. of 50 runs, 50 loops each)


In [4]:
import numpy as np
import pandas as pd
import ctypes
import timeit
from customProphet5 import CustomProphet

# Load the shared library
lib = ctypes.CDLL('./libminus_log_posterior_4.so')

## Define argument and return types for C++ functions
lib.timed_minus_log_posterior.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),  # params
    ctypes.c_size_t,  # params_size
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),  # t_scaled
    ctypes.c_size_t,  # t_scaled_size
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),  # change_points
    ctypes.c_size_t,  # change_points_size
    ctypes.c_double,  # scale_period
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),  # normalized_y
    ctypes.c_size_t,  # normalized_y_size
    ctypes.c_double,  # sigma_obs
    ctypes.c_double,  # sigma_k
    ctypes.c_double,  # sigma_m
    ctypes.c_double,  # sigma
    ctypes.c_double,  # tau
    ctypes.c_int  # num_runs
]
lib.timed_minus_log_posterior.restype = ctypes.c_double

# Load data (replace with your own data loading process)
df = pd.read_csv('peyton_manning.csv')

# Instantiate & initialize a model (replace with your model initialization process)
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

# Prepare parameters for the function call
params = np.ones((47,))  # Replace with your actual parameters

params_ctypes = np.ascontiguousarray(params, dtype=np.float64)
t_scaled_ctypes = np.ascontiguousarray(model.t_scaled, dtype=np.float64)
change_points_ctypes = np.ascontiguousarray(model.change_points, dtype=np.float64)
normalized_y_ctypes = np.ascontiguousarray(model.normalized_y, dtype=np.float64)

# Define constants (replace with your actual constants)
sigma_obs = model.sigma_obs
sigma_k = model.sigma_k
sigma_m = model.sigma_m
sigma = model.sigma
tau = model.tau

# Number of runs for timing
num_runs = 1000

# Timing Python function
def time_python_function():
    return timeit.timeit(lambda: model._minus_log_posterior(params), number=num_runs)

# Timing C++ function
def time_cpp_function():
    return lib.timed_minus_log_posterior(
        params_ctypes, len(params_ctypes),
        t_scaled_ctypes, len(t_scaled_ctypes),
        change_points_ctypes, len(change_points_ctypes),
        model.scale_period,
        normalized_y_ctypes, len(normalized_y_ctypes),
        sigma_obs,
        sigma_k,
        sigma_m,
        sigma,
        tau,
        num_runs
    )

python_time = time_python_function()
cpp_time = time_cpp_function()

# Print results
print(f"Python minus log posterior took {python_time / num_runs:.6f} seconds per run on average.")
print(f"C++ minus log posterior took {cpp_time / num_runs:.6f} seconds per run on average.")

Python minus log posterior took 0.001490 seconds per run on average.
C++ minus log posterior took 0.000414 seconds per run on average.


In [2]:
import numpy as np
import pandas as pd
import ctypes
from customProphet6 import *

# Load the shared library
lib = ctypes.CDLL('./libminus_log_posterior_6.so')

# Define argument and return types
lib.minus_log_posterior.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double
]
lib.minus_log_posterior.restype = ctypes.c_double

# Load data
df = pd.read_csv('peyton_manning.csv')

# Instantiate & initialize a model
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

params = np.ones((47,))

# Convert data to ctypes
params_ctypes = np.ascontiguousarray(params, dtype=np.float64)
t_scaled_ctypes = np.ascontiguousarray(model.t_scaled, dtype=np.float64)
change_points_ctypes = np.ascontiguousarray(model.change_points, dtype=np.float64)
normalized_y_ctypes = np.ascontiguousarray(model.normalized_y, dtype=np.float64)

mpl = model._minus_log_posterior(params)

# Call the C++ function
mlp_cpp = lib.minus_log_posterior(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau
)

print("Minus log posterior (C++):", mlp_cpp)
print("Minus log posterior (Python):", mpl)

Minus log posterior (C++): 16865146.05842033
Minus log posterior (Python): 16865146.058420338


In [7]:
%%timeit -r 50 -n 50

lib.minus_log_posterior(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau
)

349 µs ± 57.7 µs per loop (mean ± std. dev. of 50 runs, 50 loops each)


In [4]:
%%timeit -r 50 -n 50

model._minus_log_posterior(params)

The slowest run took 8.72 times longer than the fastest. This could mean that an intermediate result is being cached.
1.02 ms ± 567 µs per loop (mean ± std. dev. of 50 runs, 50 loops each)
